In [1]:
import pandas as pd
import os
import itertools
from itertools import combinations, permutations
import random
import numpy as np

## Generate names, places, locations and objects

In [2]:
name =  pd.read_csv('https://www.cs.cmu.edu/Groups/AI/util/areas/nlp/corpora/names/female.txt')[4:].sample(n=25, random_state = 88)['# List of common female names.'].values
place = ['bag','backpack', 'bin','box', 'briefcase',
         'cabinet', 'chest', 'coffer', 'chamber','locker','suitcase',
         'crate', 'tote', 'knapsack', 'hamper', 'carryall', 'drawer',
         'case', 'cupboard', 'pantry', 'safe',
         'refrigerator', 'closet']
location = ['attic', 'bedroom', 'study', 'garage', 'living room', 'basement', 'loft', 'kitchen', 'terrace', 'balcony', 'nursery',
            'conservatory', 'lounge', 'dining room', 'library',
            'farm', 'bar', 'market', 'garden', 'bank', 'office', 'factory',
            'house', 'hall', 'bathroom', 'shower room', 'storeroom', 'parking', 'squad room',
            'recreation room', 'locker room', 'belfry', 'classroom', 'barroom', 'lounge',
            'checkroom', 'guardroom', 'ballroom', 'cardroom', 'clubroom', 'boardroom',
            'dinette', 'compartment', 'reception room', 'reading room', 'smoking room', 'dance studio',
            'laundry room', 'waiting room', 'lobby']
objects = ['beet', 'broccoli', 'cabbage','carrot', 'corn', 'cucumber', 'cauliflower', 'eggplant', 'pepper', 'pumpkin', 'potato', 'tomato', 'zucchini',
         'banana', 'coconut', 'durian', 'grapefruit', 'guava', 'lemon', 'mango', 'melon', 'papaya', 'peach', 'pear', 'persimmon', 'pineapple', 'plum',
         'sweater', 'jacket', 'coat', 'vest', 'dress', 'sock', 'raincoat', 'skirt', 'blouse', 'shirt', 'tie', 'scarf', 'cap','ring', 'watch', 'cardigan', 'hoodie', 'mask', 'mitten',
         'binder', 'book', 'bottle', 'bulb', 'calculator', 'casette', 'chalk', 'pencil', 'compass', 'cork', 'crayon', 'cup', 'tape', 'fork', 'lock', 'marker', 'mug', 'notebook', 'pen', 'plate',
         'rubber', 'ruler', 'torch', 'cable',
         'knife', 'jar', 'sponge', 'towel', 'spoon', 'button', 'candle','hammer', 'key', 'magnet', 'thermometer']

## Create Story template

In [3]:
class data_smarties:
  def __init__(self, name, obj, location, place, question):
    self.name = name ##[A,B]
    self.obj = obj ## [O1,O2]
    self.location = location ## L1
    self.place = place ##P1
    self.Q = question ##['reality', 'belief', '1stA', '1stB', '2ndA', '2ndB']
  def premise(self):
    sent1 = "{} found a {} in the {}.".format(self.name[0], self.place, self.location)
    sent2 = "The label on the {} says {}.".format(self.place, self.obj[0])
    sent3 = "{} couldn’t see what was inside the {}.".format(self.name[0], self.place)
    sent4 = "{} opened the {} and found a {}.".format(self.name[0], self.place, self.obj[1])
    sent5 = "There is no {} in the {}.".format(self.obj[0], self.place)
    sent6 = "{} closed the {} and put it back.".format(self.name[0], self.place)
    sent7 = "{} entered the {} and saw the {}.".format(self.name[1], self.location, self.place)
    return [sent1, sent2, sent3, sent4, sent5, sent6, sent7]

  def prompt(self):
    story = " ".join(self.premise())
    return story

  def question(self):
    if self.Q == '1stB':
      return "After {} opened the {}, what would {} expect to find in the {}?".format(self.name[1], self.place, self.name[1], self.place)
    elif self.Q == '2ndA':
      return "After {} opened the {}, what would {} think {} would expect to find in the {}?".format(self.name[1], self.place,  self.name[0], self.name[1], self.place)
    elif self.Q == '2ndB':
      return "After {} opened the {}, what would {} think {} would expect to find in the {}?".format(self.name[1], self.place, self.name[1], self.name[0], self.place)
  def answer(self):
    if self.Q =='1stB':
      return "{}".format(self.obj[0])
    elif self.Q == '2ndA':
      return "{}".format(self.obj[0])
    elif self.Q == '2ndB':
      return "{}".format(self.obj[0])
  def qa_prompt(self):
    start = 'Answer the question based on context:\n\nContext:'
    middle = "\nQuestion:\n"
    output = start + self.prompt() + middle + self.question()
    return output
  def comp_prompt(self):
    start = "Complete the following paragraph:\n"
    if self.Q == 'reality':
      end = "In the {}, there was ".format(self.place)
    elif self.Q == 'belief':
      end = "In the {}, there was supposed to be ".format(self.place)
    elif self.Q =='1stA':
      end = "After {} opened the {}, {} would expect to find".format(self.name[1], self.place, self.name[0])
    elif self.Q == '1stB':
      end = "After {} opened the {}, {} would expect to find".format(self.name[1], self.place, self.name[1])
    elif self.Q == '2ndA':
      end = "After {} opened the {}, {} would think {} would expect to find".format(self.name[1], self.place, self.name[0], self.name[1])
    elif self.Q == '2ndB':
      end = "After {} opened the {}, {} would think {} would expect to find".format(self.name[1], self.place, self.name[1], self.name[0])
    output = start + self.prompt() + " " + end
    return output


In [4]:
class data_SallyAnne:
  def __init__(self, name, obj, location, place, question):
    self.name = name ##[A,B]
    self.obj = obj ## O1
    self.location = location ## L1
    self.place = place ## [P1,P2]
    self.Q = question ##['reality', 'belief', '1stA', '1stB', '2ndA', '2ndB']
  def premise(self):
    sent1 = "{} and {} were hanging out in the {}.".format(self.name[0], self.name[1],self.location)
    sent2 = "They saw a {} and a {}.".format(self.place[0], self.place[1])
    sent3 = "They found a {} in the {}.".format(self.obj, self.place[0])
    sent4 = "{} left the {}.".format(self.name[1], self.location)
    sent5 = "{} moved the {} to the {}.".format(self.name[0], self.obj, self.place[1])
    return [sent1, sent2, sent3, sent4, sent5]
  def prompt(self):
    story = " ".join(self.premise())
    return story
  def question(self):
    if self.Q == '1stB':
      return "After {} came back to the {}, where would {} look for the {}?".format(self.name[1], self.location, self.name[1], self.obj)
    elif self.Q == '2ndA':
      return "After {} came back to the {}, where would {} think {} would look for the {}?".format(self.name[1], self.location, self.name[0], self.name[1], self.obj)
    else:
      return "After {} came back to the {}, where would {} think {} would look for the {}?".format(self.name[1], self.location, self.name[1], self.name[0], self.obj)
  def answer(self):
    if self.Q == '1stB':
      return '{}'.format(self.place[0])
    elif self.Q == '2ndA':
      return '{}'.format(self.place[0])
    elif self.Q == '2ndB':
      return '{}'.format(self.place[0])
  def qa_prompt(self):
    start = 'Answer the question based on context:\n\nContext:'
    middle = "\nQuestion:\n"
    output = start + self.prompt() + middle + self.question()
    return output
  def comp_prompt(self):
    start = "Complete the following paragraph:\n"
    if self.Q == 'reality':
      end = "Currently the {} is in".format(self.obj)
    elif self.Q == 'belief':
      end = "Previously the {} was in".format(self.obj)
    elif self.Q =='1stA':
      end = "After {} came back to the {}, {} would look for the {} in".format(self.name[1], self.location, self.name[0], self.obj)
    elif self.Q == '1stB':
      end = "After {} came back to the {}, {} would look for the {} in".format(self.name[1], self.location, self.name[1], self.obj)
    elif self.Q == '2ndA':
      end = "After {} came back to the {}, {} would think {} would look for the {} in".format(self.name[1], self.location, self.name[0], self.name[1], self.obj)
    elif self.Q == '2ndB':
      end = "After {} came back to the {}, {} would think {} would look for the {} in".format(self.name[1], self.location, self.name[1], self.name[0], self.obj)
    output = start + self.prompt() + " " + end
    return output

## Generate stories

In [5]:
output_list_smarties = []
random.seed(42)
name_list = random.choices(list(permutations(name, 2)), k = 30)
object_list = random.choices(list(permutations(objects, 2)), k = 30)
for i in range(0, 30):
  n = name_list[i]
  p = place[i%len(place)]
  o = object_list[i]
  l = location[i%len(location)]
  for que in ['1stB', '2ndA', '2ndB']:
    smarties = data_smarties(n, o, l, p, que)
    output_list_smarties.append([smarties.prompt(), smarties.question(), smarties.answer(), que, i+1,
                                 smarties.qa_prompt(), smarties.comp_prompt()])
df_smarties = pd.DataFrame(output_list_smarties, columns = ['prompt', 'question', 'short_answer', 'question_type', 'story_index',
                                                            'qa_prompt', 'comp_prompt'])

In [6]:
output_list_sallyanne = []
random.seed(42)
name_list = random.choices(list(permutations(name, 2)), k = 30)
place_list = random.choices(list(permutations(place, 2))*5, k = 30)
for i in range(0, 30):
  n = name_list[i]
  p = place_list[i]
  o = object_list[i][0]
  l = location[i%len(location)]
  for que in ['1stB', '2ndA', '2ndB']:
    sally_anne = data_SallyAnne(n, o, l, p, que)
    output_list_sallyanne.append([sally_anne.prompt(), sally_anne.question(), sally_anne.answer(), que, i+1,
                                  sally_anne.qa_prompt(), sally_anne.comp_prompt()])
df_sallyanne = pd.DataFrame(output_list_sallyanne, columns = ['prompt', 'question', 'short_answer', 'question_type', 'story_index',
                                                              'qa_prompt', 'comp_prompt'])


In [10]:
df_smarties.to_csv('Smarties_prompt.csv', index=False)

In [11]:
df_sallyanne.to_csv('Sally-Anne_prompt.csv', index=False)

,prompt,question,short_answer,question_type,story_index,qa_prompt,comp_prompt
0,Neila and Juanita were hanging out in the atti...,"After Juanita came back to the attic, where wo...",bag,1stB,1,Answer the question based on context:\n\nConte...,Complete the following paragraph:\nNeila and J...
1,Neila and Juanita were hanging out in the atti...,"After Juanita came back to the attic, where wo...",bag,2ndA,1,Answer the question based on context:\n\nConte...,Complete the following paragraph:\nNeila and J...
2,Neila and Juanita were hanging out in the atti...,"After Juanita came back to the attic, where wo...",bag,2ndB,1,Answer the question based on context:\n\nConte...,Complete the following paragraph:\nNeila and J...
3,Sabra and Hillary were hanging out in the bedr...,"After Hillary came back to the bedroom, where ...",hamper,1stB,2,Answer the question based on context:\n\nConte...,Complete the following paragraph:\nSabra and H...
4,Sabra and Hillary were hanging out in the bedr...,"After Hillary came back to the bedroom, where ...",hamper,2ndA,2,Answer the question based on context:\n\nConte...,Complete the following paragraph:\nSabra and H...
